<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [1]:
!pip install sqlalchemy==1.3.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 117.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... one
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp312-cp312-linux_x86_64.whl size=1160111 sha256=30fda11cac2d8201236fca3eeba8185a8c9668438538686140b92ece97005069
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/b3/1c/42/0e26b8d512adc6bce10ff71a05229366b4ccec641cd3b42111
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.37
    Uninstalling SQLAlchemy-2.0.37:
      Successfully uninstalled SQLAlchemy-2.0.37
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterhub 5.2.1 requires SQLAlchemy>=1.4.1, but you have sqlalchemy 1.3.9 which is incompatible.


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
!pip install ipython-sql
!pip install ipython-sql prettytable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 112.8 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [4]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [8]:
!pip install -q pandas

In [9]:
%sql sqlite:///my_data1.db

In [10]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [11]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [12]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [18]:
%%sql
-- Unique launch site names
SELECT DISTINCT "Launch_Site" AS launch_site
FROM SPACEXTABLE
ORDER BY 1;

 * sqlite:///my_data1.db
Done.


launch_site
CCAFS LC-40
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E


In [19]:
%%sql
SELECT 
  CASE 
    WHEN "Launch_Site" IN ('CCAFS LC-40','CCAFS SLC-40') THEN 'CCAFS SLC-40'
    ELSE "Launch_Site"
  END AS launch_site_clean
FROM SPACEXTABLE
GROUP BY launch_site_clean
ORDER BY 1;

 * sqlite:///my_data1.db
Done.


launch_site_clean
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E


In [21]:
%%sql
DROP VIEW IF EXISTS launches_clean;

CREATE VIEW launches_clean AS
SELECT
  CASE 
    WHEN "Launch_Site" IN ('CCAFS LC-40','CCAFS SLC-40') THEN 'CCAFS SLC-40'
    ELSE "Launch_Site"
  END AS "Launch_Site",
  "Date",
  "FlightNumber",
  "Booster_Version",
  "Payload_Mass_kg",
  "Orbit",
  "Customer",
  "Mission_Outcome",
  "Landing_Outcome"
FROM SPACEXTABLE;


 * sqlite:///my_data1.db
Done.
Done.


[]

In [22]:
%%sql
SELECT "Launch_Site", COUNT(*) AS n
FROM launches_clean
GROUP BY 1
ORDER BY n DESC;

 * sqlite:///my_data1.db
Done.


Launch_Site,n
CCAFS SLC-40,60
KSC LC-39A,25
VAFB SLC-4E,16



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [24]:
%%sql
SELECT *
FROM launches_clean
WHERE "Launch_Site" LIKE 'CCA%'
ORDER BY "Date" ASC
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Launch_Site,Date,"""FlightNumber""",Booster_Version,"""Payload_Mass_kg""",Orbit,Customer,Mission_Outcome,Landing_Outcome
CCAFS SLC-40,2010-06-04,FlightNumber,F9 v1.0 B0003,Payload_Mass_kg,LEO,SpaceX,Success,Failure (parachute)
CCAFS SLC-40,2010-12-08,FlightNumber,F9 v1.0 B0004,Payload_Mass_kg,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
CCAFS SLC-40,2012-05-22,FlightNumber,F9 v1.0 B0005,Payload_Mass_kg,LEO (ISS),NASA (COTS),Success,No attempt
CCAFS SLC-40,2012-10-08,FlightNumber,F9 v1.0 B0006,Payload_Mass_kg,LEO (ISS),NASA (CRS),Success,No attempt
CCAFS SLC-40,2013-03-01,FlightNumber,F9 v1.0 B0007,Payload_Mass_kg,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [32]:
%%sql
DROP VIEW IF EXISTS launches_clean;

CREATE VIEW launches_clean AS
SELECT
  CASE 
    WHEN "Launch_Site" IN ('CCAFS LC-40','CCAFS SLC-40') THEN 'CCAFS SLC-40'
    ELSE "Launch_Site"
  END AS "Launch_Site",
  "Date",
  "FlightNumber",
  "Booster_Version",
  "PAYLOAD_MASS__KG_" AS "PAYLOAD_MASS__KG_",   -- <- pakai nama kolom yang benar
  "Orbit",
  "Customer",
  "Mission_Outcome",
  "Landing_Outcome"
FROM SPACEXTABLE;


 * sqlite:///my_data1.db
Done.
Done.


[]

In [34]:
%%sql
SELECT "PAYLOAD_MASS__KG_"
FROM launches_clean
WHERE "Customer" LIKE '%NASA (CRS)%'
LIMIT 5;


 * sqlite:///my_data1.db
Done.


PAYLOAD_MASS__KG_
500
677
2296
2216
2395


In [35]:
%%sql
-- Total payload mass (kg) for NASA (CRS)
SELECT 
  'NASA (CRS)' AS customer,
  SUM(CAST("PAYLOAD_MASS__KG_" AS REAL)) AS total_payload_kg
FROM launches_clean
WHERE "Customer" LIKE '%NASA (CRS)%';

 * sqlite:///my_data1.db
Done.


customer,total_payload_kg
NASA (CRS),48213.0


In [36]:
%%sql
-- Sanity check: show a few NASA (CRS) rows with payloads
SELECT "Date","Launch_Site","PAYLOAD_MASS__KG_"
FROM launches_clean
WHERE "Customer" LIKE '%NASA (CRS)%'
ORDER BY "Date"
LIMIT 10;

 * sqlite:///my_data1.db
Done.


Date,Launch_Site,PAYLOAD_MASS__KG_
2012-10-08,CCAFS SLC-40,500
2013-03-01,CCAFS SLC-40,677
2014-04-18,CCAFS SLC-40,2296
2014-09-21,CCAFS SLC-40,2216
2015-01-10,CCAFS SLC-40,2395
2015-04-14,CCAFS SLC-40,1898
2015-06-28,CCAFS SLC-40,1952
2016-04-08,CCAFS SLC-40,3136
2016-07-18,CCAFS SLC-40,2257
2017-02-19,KSC LC-39A,2490


In [37]:
%%sql
-- Compare totals by NASA variants (optional)
SELECT "Customer",
       SUM(CAST("PAYLOAD_MASS__KG_" AS REAL)) AS total_payload_kg
FROM launches_clean
WHERE "Customer" LIKE '%NASA%'
GROUP BY "Customer"
ORDER BY total_payload_kg DESC;


 * sqlite:///my_data1.db
Done.


Customer,total_payload_kg
NASA (CRS),45596.0
NASA (CCDev),12530.0
NASA (CCP),12500.0
NASA (CCD),12055.0
NASA (CTS),12050.0
Iridium Communications GFZ ‚ NASA,6460.0
"NASA (CRS), Kacific 1",2617.0
NASA / NOAA / ESA / EUMETSAT,1192.0
U.S. Air Force NASA NOAA,570.0
NASA (LSP) NOAA CNES,553.0


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [38]:
%%sql
-- Average payload mass (kg) for booster version F9 v1.1
SELECT 
  ROUND(AVG(CAST("PAYLOAD_MASS__KG_" AS REAL)), 2) AS avg_payload_kg
FROM launches_clean
WHERE "Booster_Version" LIKE 'F9 v1.1%';


 * sqlite:///my_data1.db
Done.


avg_payload_kg
2534.67


In [39]:
%%sql
-- How many rows used to compute the average?
SELECT 
  COUNT(*) AS n_rows,
  MIN(CAST("PAYLOAD_MASS__KG_" AS REAL)) AS min_kg,
  MAX(CAST("PAYLOAD_MASS__KG_" AS REAL)) AS max_kg
FROM launches_clean
WHERE "Booster_Version" LIKE 'F9 v1.1%';


 * sqlite:///my_data1.db
Done.


n_rows,min_kg,max_kg
15,500.0,4707.0


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [40]:
%%sql
-- First date when landing was successful on a ground pad
SELECT MIN("Date") AS first_success_ground_pad
FROM launches_clean
WHERE "Landing_Outcome" LIKE 'Success (ground pad)%';


 * sqlite:///my_data1.db
Done.


first_success_ground_pad
2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [41]:
%%sql
-- Boosters with successful drone ship landings and payload mass (4000, 6000)
SELECT DISTINCT "Booster_Version" AS booster
FROM launches_clean
WHERE "Landing_Outcome" LIKE 'Success (drone ship)%'
  AND CAST("PAYLOAD_MASS__KG_" AS REAL) > 4000
  AND CAST("PAYLOAD_MASS__KG_" AS REAL) < 6000
ORDER BY booster;


 * sqlite:///my_data1.db
Done.


booster
F9 FT B1021.2
F9 FT B1031.2
F9 FT B1022
F9 FT B1026


### Task 7




##### List the total number of successful and failure mission outcomes


In [42]:
%%sql
-- Total counts of mission outcomes (Success vs Failure)
SELECT
  SUM(CASE WHEN "Mission_Outcome" LIKE 'Success%' THEN 1 ELSE 0 END) AS success_count,
  SUM(CASE WHEN "Mission_Outcome" LIKE 'Failure%' THEN 1 ELSE 0 END) AS failure_count
FROM launches_clean;


 * sqlite:///my_data1.db
Done.


success_count,failure_count
100,1


In [43]:
%%sql
-- See all mission outcome variants + counts
SELECT "Mission_Outcome", COUNT(*) AS n
FROM launches_clean
GROUP BY "Mission_Outcome"
ORDER BY n DESC;


 * sqlite:///my_data1.db
Done.


Mission_Outcome,n
Success,98
Success (payload status unclear),1
Success,1
Failure (in flight),1


In [44]:
%%sql
-- Clean grouping + overall rates
WITH outcomes AS (
  SELECT CASE
           WHEN TRIM("Mission_Outcome") LIKE 'Success%' THEN 'Success'
           WHEN TRIM("Mission_Outcome") LIKE 'Failure%' THEN 'Failure'
           ELSE 'Other'
         END AS outcome_group
  FROM launches_clean
)
SELECT 
  outcome_group,
  COUNT(*) AS n,
  ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM outcomes), 2) AS pct
FROM outcomes
GROUP BY outcome_group
ORDER BY n DESC;


 * sqlite:///my_data1.db
Done.


outcome_group,n,pct
Success,100,99.01
Failure,1,0.99


### Task 8



##### List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.


In [45]:
%%sql
-- Booster versions that carried the maximum payload mass (kg)
SELECT DISTINCT "Booster_Version"
FROM launches_clean
WHERE CAST("PAYLOAD_MASS__KG_" AS REAL) = (
    SELECT MAX(CAST("PAYLOAD_MASS__KG_" AS REAL))
    FROM launches_clean
);


 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [46]:
%%sql
-- Records in 2015 with FAILED drone-ship landings, showing month name, booster, and site
SELECT 
  CASE CAST(SUBSTR("Date", 6, 2) AS INTEGER)
    WHEN 1  THEN 'January'
    WHEN 2  THEN 'February'
    WHEN 3  THEN 'March'
    WHEN 4  THEN 'April'
    WHEN 5  THEN 'May'
    WHEN 6  THEN 'June'
    WHEN 7  THEN 'July'
    WHEN 8  THEN 'August'
    WHEN 9  THEN 'September'
    WHEN 10 THEN 'October'
    WHEN 11 THEN 'November'
    WHEN 12 THEN 'December'
  END AS month_name,
  "Landing_Outcome",
  "Booster_Version",
  "Launch_Site",
  "Date"
FROM launches_clean
WHERE SUBSTR("Date", 1, 4) = '2015'
  AND "Landing_Outcome" LIKE 'Failure (drone ship)%'
ORDER BY CAST(SUBSTR("Date", 6, 2) AS INTEGER), "Date";


 * sqlite:///my_data1.db
Done.


month_name,Landing_Outcome,Booster_Version,Launch_Site,Date
January,Failure (drone ship),F9 v1.1 B1012,CCAFS SLC-40,2015-01-10
April,Failure (drone ship),F9 v1.1 B1015,CCAFS SLC-40,2015-04-14


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [47]:
%%sql
-- Rank counts of landing outcomes in the given date range
SELECT 
  "Landing_Outcome",
  COUNT(*) AS n
FROM launches_clean
WHERE "Date" >= '2010-06-04'
  AND "Date" <= '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY n DESC, "Landing_Outcome";


 * sqlite:///my_data1.db
Done.


Landing_Outcome,n
No attempt,10
Failure (drone ship),5
Success (drone ship),5
Controlled (ocean),3
Success (ground pad),3
Failure (parachute),2
Uncontrolled (ocean),2
Precluded (drone ship),1


In [48]:
%%sql
-- Total launches to geosynchronous orbit (GTO)
SELECT COUNT(*) AS n_gto
FROM launches_clean
WHERE "Orbit" = 'GTO';


 * sqlite:///my_data1.db
Done.


n_gto
30


In [49]:
%%sql
-- Total missions with successful landing on a drone ship
SELECT COUNT(*) AS n_success_drone_ship
FROM launches_clean
WHERE "Landing_Outcome" LIKE 'Success (drone ship)%';


 * sqlite:///my_data1.db
Done.


n_success_drone_ship
14


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
